In [ ]:
#安裝相關函式庫
#!pip install requests

In [ ]:
import requests
from lxml import etree


In [ ]:
#可以從瀏覽器的開發者工具取得 request 的 header 資訊
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
    }

In [ ]:
#請求網頁
def requestHtml(url):
  s = requests.Session()
  r = s.get(url, headers=headers_Get)
  r.encoding = r.apparent_encoding
  #print(r.text)
  return etree.HTML(r.text)

In [ ]:
#限定搜尋結果(指定網站)
url = 'https://www.google.com/search?q=%E7%83%8F%E5%85%8B%E8%98%AD+site%3Altn.com.tw&oq=%E7%83%8F%E5%85%8B%E8%98%AD+site%3Altn.com.tw&aqs=chrome..69i57.9993j0j4&sourceid=chrome&ie=UTF-8'
dom = requestHtml(url)

links = dom.xpath('//h3/parent::a/@href')
titles = dom.xpath('//h3/text()')

print(links)
print(titles)

['https://news.ltn.com.tw/news/world/breakingnews/3893838', 'https://news.ltn.com.tw/news/world/breakingnews/3892829', 'https://news.ltn.com.tw/news/world/breakingnews/3893493', 'https://news.ltn.com.tw/news/world/breakingnews/3892625', 'https://news.ltn.com.tw/news/world/breakingnews/3892189', 'https://news.ltn.com.tw/news/world/breakingnews/3892538', 'https://ec.ltn.com.tw/article/breakingnews/3893662', 'https://news.ltn.com.tw/news/world/breakingnews/3892607', 'https://news.ltn.com.tw/news/world/breakingnews/3892596', 'https://news.ltn.com.tw/news/world/breakingnews/3891350']
['普廷堅持5/9取勝《彭博》：俄軍恐在烏克蘭再次慘敗 - 自由時報', '德前總統籲「放棄舒適」挺烏克蘭過半數德國人不買單 - 自由時報', '烏克蘭2飛官獲釋情報局局長親自接見 - 自由時報', '烏克蘭稱2枚飛彈擊中「莫斯科號」 俄軍證實船艦嚴重受損', '通話曝光！ 俄軍強姦烏克蘭女性頻傳老婆還提醒「記得戴套」', '烏東大戰一觸即發！ 歐盟火力支援烏克蘭週三再批准158億', '抗俄開銷澤倫斯基：烏克蘭每月要燒掉2千億 - 自由財經', '烏克蘭再傳戰果「海王星飛彈」擊中俄軍艦「莫斯科號」 - 國際', '拜登加碼8億美元軍援挺烏克蘭首度提供榴彈砲 - 自由時報', '烏親俄領袖被捕拜登控普廷試圖滅絕烏克蘭人（不斷更新 ...']


In [ ]:
#下一頁 
#//span[text()='下一頁']/parent::a/@href
#//span[contains(text(),'下一頁')]
nextPage = dom.xpath('//a[@id="pnnext"]/@href')
print(nextPage)

['/search?q=%E7%83%8F%E5%85%8B%E8%98%AD+site:ltn.com.tw&ei=JLJYYoTeNcnSsAfVjpJQ&start=10&sa=N&ved=2ahUKEwiEgI2i3pT3AhVJKewKHVWHBAoQ8NMDegQIARBP']


In [ ]:
#這樣只能一頁
if nextPage:
  dom = requestHtml('https://www.google.com' + nextPage[0])

links += dom.xpath('//h3/parent::a/@href')
titles += dom.xpath('//h3/text()')

print(len(links))
print(len(titles))

20
20


In [ ]:
#當有下一頁時，就一直爬取
while nextPage:
  dom = requestHtml('https://www.google.com' + nextPage[0])

  links += dom.xpath('//h3/parent::a/@href')
  titles += dom.xpath('//h3/text()')

  print(len(links))
  print(len(titles))

  nextPage = dom.xpath('//a[@id="pnnext"]/@href')


In [ ]:
for link in links:
  print(link)

https://news.ltn.com.tw/news/world/breakingnews/3893838
https://news.ltn.com.tw/news/world/breakingnews/3892829
https://news.ltn.com.tw/news/world/breakingnews/3893493
https://news.ltn.com.tw/news/world/breakingnews/3892625
https://news.ltn.com.tw/news/world/breakingnews/3892189
https://news.ltn.com.tw/news/world/breakingnews/3892538
https://ec.ltn.com.tw/article/breakingnews/3893662
https://news.ltn.com.tw/news/world/breakingnews/3892607
https://news.ltn.com.tw/news/world/breakingnews/3892596
https://news.ltn.com.tw/news/world/breakingnews/3891350
https://news.ltn.com.tw/news/world/breakingnews/3893019
https://news.ltn.com.tw/news/world/breakingnews/3890862
https://news.ltn.com.tw/news/world/breakingnews/3892505
https://news.ltn.com.tw/news/world/breakingnews/3890150
https://news.ltn.com.tw/news/world/breakingnews/3892573
https://news.ltn.com.tw/news/world/breakingnews/3890429
https://ec.ltn.com.tw/article/breakingnews/3893256
https://news.ltn.com.tw/news/world/breakingnews/3893840
ht

In [ ]:
print(len(links))
#去除重複
links = list(dict.fromkeys(links))
print(len(links))

365
365


In [ ]:
print(type(links))
print(links[0])

dom = requestHtml(links[0])
#取得標題、內文、發文日期、圖檔連結
title = dom.xpath('//h1/text()')
image = dom.xpath('//a[@class="image-popup-vertical-fit"]/@href')
publish_date = dom.xpath('//span[@class="time"]/text()')
content = dom.xpath('//span[@class="time"]/following-sibling::p/text()')
#連接 list 內的值，使用斷行 \n
content = '\n'.join(content)
#組成dict
article = {
    "title": title,
    "image": image,
    "publish_date": publish_date,
    "content": content
}

print(article)

<class 'list'>
https://news.ltn.com.tw/news/world/breakingnews/3893838
{'title': ['普廷堅持5/9取勝 《彭博》：俄軍恐在烏克蘭再次慘敗'], 'image': ['https://img.ltn.com.tw/Upload/news/600/2022/04/14/3893838_1_1.jpg'], 'publish_date': [<Element span at 0x7fce4d075b40>], 'content': '首次上稿 04-14 23:44\n更新時間 04-15 06:23\n〔編譯楊芙宜／台北報導〕《彭博》報導，如果俄國仍試圖爭取在5月9日前取得在烏克蘭戰爭的某種勝利，可能重蹈之前失誤，再次慘敗；據專家分析，受限於天氣條件，需要後勤準備工作、建立空中主導優勢、以及重新部署在基輔附近受挫的俄國部隊，需花更多時間才能奪回仍在烏克蘭控制下的頓巴斯（Donbas）其他地區。\n請繼續往下閱讀...\n報導指出，如果俄羅斯在春季地面泥濘的高峰期將部隊投入戰鬥，那將很難達到目的；屆時地面濕軟，卡車和移動火砲離開道路就可能陷住，這種情況在基輔以北曾發生過，導致一些俄國車隊滯留，易受攻擊。\n一位不具名的莫斯科分析師表示，這個季節將在5月份結束，但可能來不及讓俄羅斯總統普廷在5月9日「勝利日」遊行前取得重大勝利。5月9日是俄羅斯慶祝在二次世界大戰蘇聯擊敗納粹德國的勝利日，莫斯科傳統會在當日舉行大閱兵。\n知情歐洲和北約外交官表示，普廷下令發起攻勢之前，烏克蘭盟友認為有兩週的時間窗口，來悄悄調動重型武器如坦克等。歐盟外交和安全政策高級代表波瑞爾（Josep Borrell）13日表示，「未來幾週將是決定性的」。歐盟已決議再另撥5億歐元（約新台幣160.14億元），向烏克蘭提供軍事援助；這使歐盟軍援烏克蘭累計達到15億歐元（約新台幣480.45億元）。\n俄羅斯官媒14日報導，其黑海艦隊旗艦「莫斯科號」因彈藥爆炸「嚴重受損」。烏克蘭南部軍事指揮部證實，俄軍這艘巡洋艦莫斯科號就是遭到烏軍2枚飛彈擊中而起火燃燒。\n普廷似乎已認識到俄軍遭遇到慘烈傷亡，速度和傷亡之間的取捨是一個問題。普廷12日記者會上提到，自己經常被問到是否無法更快結束戰爭，「這是可能的。這取決於軍事行動的強度，不幸的

In [ ]:
def composeArticle(dom, link):
  #取得標題、內文、發文日期、圖檔連結
  title = dom.xpath('//h1/text()')
  image = dom.xpath('//a[@class="image-popup-vertical-fit"]/@href')
  publish_date = dom.xpath('//span[@class="time"]/text()')
  content = dom.xpath('//span[@class="time"]/following-sibling::p/text()')
  #連接 list 內的值，使用斷行 \n
  content = '\n'.join(content)

  article_image = ''
  if image:
    article_image = image[0]
  #組成dict
  return {
    "title": title[0],
    "image": article_image,
    "publish_date": publish_date[0],
    "content": content,
    "link": link
  }

In [ ]:
from random import randint
from time import sleep

In [ ]:
articles = []
for link in links[:10]:
  dom = requestHtml(link)

  article = composeArticle(dom, link)
  print(article)
  articles.append(article)

  sleep(randint(1,3))

{'title': '普廷堅持5/9取勝 《彭博》：俄軍恐在烏克蘭再次慘敗', 'image': 'https://img.ltn.com.tw/Upload/news/600/2022/04/14/3893838_1_1.jpg', 'publish_date': '\n    2022/04/15 06:23', 'content': '首次上稿 04-14 23:44\n更新時間 04-15 06:23\n〔編譯楊芙宜／台北報導〕《彭博》報導，如果俄國仍試圖爭取在5月9日前取得在烏克蘭戰爭的某種勝利，可能重蹈之前失誤，再次慘敗；據專家分析，受限於天氣條件，需要後勤準備工作、建立空中主導優勢、以及重新部署在基輔附近受挫的俄國部隊，需花更多時間才能奪回仍在烏克蘭控制下的頓巴斯（Donbas）其他地區。\n請繼續往下閱讀...\n報導指出，如果俄羅斯在春季地面泥濘的高峰期將部隊投入戰鬥，那將很難達到目的；屆時地面濕軟，卡車和移動火砲離開道路就可能陷住，這種情況在基輔以北曾發生過，導致一些俄國車隊滯留，易受攻擊。\n一位不具名的莫斯科分析師表示，這個季節將在5月份結束，但可能來不及讓俄羅斯總統普廷在5月9日「勝利日」遊行前取得重大勝利。5月9日是俄羅斯慶祝在二次世界大戰蘇聯擊敗納粹德國的勝利日，莫斯科傳統會在當日舉行大閱兵。\n知情歐洲和北約外交官表示，普廷下令發起攻勢之前，烏克蘭盟友認為有兩週的時間窗口，來悄悄調動重型武器如坦克等。歐盟外交和安全政策高級代表波瑞爾（Josep Borrell）13日表示，「未來幾週將是決定性的」。歐盟已決議再另撥5億歐元（約新台幣160.14億元），向烏克蘭提供軍事援助；這使歐盟軍援烏克蘭累計達到15億歐元（約新台幣480.45億元）。\n俄羅斯官媒14日報導，其黑海艦隊旗艦「莫斯科號」因彈藥爆炸「嚴重受損」。烏克蘭南部軍事指揮部證實，俄軍這艘巡洋艦莫斯科號就是遭到烏軍2枚飛彈擊中而起火燃燒。\n普廷似乎已認識到俄軍遭遇到慘烈傷亡，速度和傷亡之間的取捨是一個問題。普廷12日記者會上提到，自己經常被問到是否無法更快結束戰爭，「這是可能的。這取決於軍事行動的強度，不幸的是，軍事行動的強度總牽連損失」、「我們的任務是實現既定目標，並把損失降至最低」。\n報導說，各方對俄軍迄今戰亡人數估計差異相當大，俄羅斯本身僅估計2343名士兵死亡，美國

In [ ]:
import csv

csv_file = '自由時報_烏克蘭.csv'
csv_columns = ['title', 'image', 'publish_date', 'content', 'link']
try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in articles:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [ ]:
#將 dict 轉換成 json 字串
import json
# Serializing json  
json_object = json.dumps(articles, indent = 4, ensure_ascii=False).encode('utf-8').decode('utf-8')
print(json_object)

[
    {
        "title": "普廷堅持5/9取勝 《彭博》：俄軍恐在烏克蘭再次慘敗",
        "image": "https://img.ltn.com.tw/Upload/news/600/2022/04/14/3893838_1_1.jpg",
        "publish_date": "\n    2022/04/15 06:23",
        "content": "首次上稿 04-14 23:44\n更新時間 04-15 06:23\n〔編譯楊芙宜／台北報導〕《彭博》報導，如果俄國仍試圖爭取在5月9日前取得在烏克蘭戰爭的某種勝利，可能重蹈之前失誤，再次慘敗；據專家分析，受限於天氣條件，需要後勤準備工作、建立空中主導優勢、以及重新部署在基輔附近受挫的俄國部隊，需花更多時間才能奪回仍在烏克蘭控制下的頓巴斯（Donbas）其他地區。\n請繼續往下閱讀...\n報導指出，如果俄羅斯在春季地面泥濘的高峰期將部隊投入戰鬥，那將很難達到目的；屆時地面濕軟，卡車和移動火砲離開道路就可能陷住，這種情況在基輔以北曾發生過，導致一些俄國車隊滯留，易受攻擊。\n一位不具名的莫斯科分析師表示，這個季節將在5月份結束，但可能來不及讓俄羅斯總統普廷在5月9日「勝利日」遊行前取得重大勝利。5月9日是俄羅斯慶祝在二次世界大戰蘇聯擊敗納粹德國的勝利日，莫斯科傳統會在當日舉行大閱兵。\n知情歐洲和北約外交官表示，普廷下令發起攻勢之前，烏克蘭盟友認為有兩週的時間窗口，來悄悄調動重型武器如坦克等。歐盟外交和安全政策高級代表波瑞爾（Josep Borrell）13日表示，「未來幾週將是決定性的」。歐盟已決議再另撥5億歐元（約新台幣160.14億元），向烏克蘭提供軍事援助；這使歐盟軍援烏克蘭累計達到15億歐元（約新台幣480.45億元）。\n俄羅斯官媒14日報導，其黑海艦隊旗艦「莫斯科號」因彈藥爆炸「嚴重受損」。烏克蘭南部軍事指揮部證實，俄軍這艘巡洋艦莫斯科號就是遭到烏軍2枚飛彈擊中而起火燃燒。\n普廷似乎已認識到俄軍遭遇到慘烈傷亡，速度和傷亡之間的取捨是一個問題。普廷12日記者會上提到，自己經常被問到是否無法更快結束戰爭，「這是可能的。這取決於軍事行動的強度，不幸的是，軍事行動的強度總牽連損失」、「我們的任務是實現既定目標，並把損失降至最低」。\n報導說，

In [ ]:
with open("自由時報_烏克蘭.json", "w", encoding='utf-8') as outfile:
    json.dump(articles, outfile, ensure_ascii=False)